# QM based correction of auranofin MM potential

This work is application of the force field correction technique published in https://doi.org/10.1021/acs.jcim.1c00651 on auranofin.
The QM methods were updated (mostly to handl gold properly) according to https://pubs.acs.org/doi/full/10.1021/ct400980w (thanks, M.Krupička).
The computational protocol is:
* Identify twistable torsions in the molecule
* Run replica exchange metadynamics, each replica having one of the torsions as its CV; this is believed to make quick, exhaustive, and reasonably realistic sampling of the conformational space
* Cluster the resulting trajectories to end up with at most few dozens of representative samples
* Run QM geometry optimization of all these samples while keeping the twistable torsions constrained
* Record final single point energies of the results
* Further relax the structures (with constrained torsions) in MM and record the energies
* The difference of the QM and MM energies can be used to add bias potential to MD (e.g. with https://www.plumed.org/doc-v2.8/user-doc/html/_p_r_o_p_e_r_t_y_m_a_p.html, based on https://doi.org/10.1063/1.3660208) to get a more realistic potential

The whole calculation is implemented in a fork of PMCVFF service: 
https://gitlab.ics.muni.cz/3086/magicforcefield-pipeline/-/blob/auranofin/auranofin.ipynb
and it requires the environment (e.g. https://pmcvff-correction.cerit-sc.cz) up and running.

This notebooks visualizes the resulting structures and gives elemetary insight to the energies.
Right now it is based on intermediate results (QM optimization of all the clusters has not converged) therefore it is illustrative only.




In [ ]:
import mdtraj as md
import nglview as nv
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
# energies extracted from Orca/b2plyp calculation
# normalized to start from zero

b2plyp = {'outCluster11': 10.136624066159886,
 'outCluster0': 9.072813808019363,
 'outCluster12': 9.106465600921283,
 'outCluster10': 17.155652562538663,
 'outCluster19': 73.52046247424472,
 'outCluster8': 13.08088474601164,
 'outCluster4': 22.67910175872164,
 'outCluster15': 37.83981410708109,
 'outCluster9': 6.627769685373793,
 'outCluster6': 21.272211152680786,
 'outCluster21': 35.17584614780066,
 'outCluster13': 47.309419875242966,
 'outCluster1': 1.059961783205897,
 'outCluster22': 68.64769656703379,
 'outCluster5': 32.054600111899234,
 'outCluster17': 27.47243619098727,
 'outCluster7': 0.0,
 'outCluster20': 53.954926299404434,
 'outCluster24': 20.332655053301213,
 'outCluster3': 46.15568595422845,
 'outCluster23': 20.98690716206865,
 'outCluster2': 41.01026540915279,
 'outCluster16': 15.871336865120757,
 'outCluster14': 17.934761328701146,
 'outCluster18': 22.80218470630036}

In [ ]:
# energies of the output of b2plyp after constrained relaxation in MM 

mm = {'outCluster11': 162.254975,
 'outCluster0': 702.422943,
 'outCluster12': 0.0967409999999802,
 'outCluster10': 13.018234999999976,
 'outCluster19': 21.27387999999999,
 'outCluster8': 0.2446289999999749,
 'outCluster4': 17.112319999999983,
 'outCluster15': 429.327973,
 'outCluster9': 0.28836099999998055,
 'outCluster6': 197.319031,
 'outCluster21': 0.2778319999999894,
 'outCluster13': 2181.817719,
 'outCluster1': 393.740204,
 'outCluster22': 17.075301999999994,
 'outCluster5': 714.0093689999999,
 'outCluster17': 0.0,
 'outCluster7': 17.12570199999999,
 'outCluster20': 2067.4908140000002,
 'outCluster24': 2.6863099999999918,
 'outCluster3': 0.17771899999999619,
 'outCluster23': 0.37109399999999937,
 'outCluster2': 17.76902799999999,
 'outCluster16': 301.022156,
 'outCluster14': 0.17134099999998398,
 'outCluster18': 0.07833899999999971}

In [ ]:
# energy deltas of the MM constrained relaxation

mmdelta = dict()
for c in mm.keys():
    xvg = np.loadtxt(f'{c}.xvg',comments=list('@#'))
    mmdelta[c] = xvg[-1,1]-xvg[0,1]
mmdelta

In [ ]:
"""
Align (on the C-S-Au-P torsion) and display the structures

Coloring:
    - yellow: input structure for b2plyp optimization
    - blue: output of b2plyp optimization
    - green: further constrained relaxation in MM
"""

trs_cl, trs_qm, trs_mm = [],[],[]
ref = md.load_pdb('outCluster0.pdb')

rmsd_clqm, rmsd_qmmm = [],[]

idx = [9,10,43,51]

for c in mm.keys():
    trqm = md.load(f'{c}_opt.pdb')
    trcl = md.load(f'{c}.pdb')
    trmm = md.load(f'{c}_after_em1.gro')
    
    rmsd_clqm.append(md.rmsd(trqm,trcl[0])[0])
    rmsd_qmmm.append(md.rmsd(trmm,trqm[0])[0])
    
    trqm.superpose(ref,atom_indices=idx)
    trs_qm.append(trqm)
    trcl.superpose(ref,atom_indices=idx)
    trs_cl.append(trcl)
    trmm.superpose(ref,atom_indices=idx)
    trs_mm.append(trmm)
    

    
trqm = md.join(trs_qm)
trcl = md.join(trs_cl)
trmm = md.join(trs_mm)

In [ ]:

r=.5
v = nv.NGLWidget()
v.clear()

cqm = v.add_trajectory(trqm)
cqm.clear()
cqm.add_licorice(color = 'blue')
cqm.add_representation('spacefill',selection=[10,43,51],radius=r)

ccl = v.add_trajectory(trcl)
ccl.clear()
ccl.add_licorice(color = 'yellow')
ccl.add_representation('spacefill',selection=[10,43,51],radius=r)

cmm = v.add_trajectory(trmm)
cmm.clear()
cmm.add_licorice(color = 'green')
cmm.add_representation('spacefill',selection=[10,43,51],radius=r)


v

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(b2plyp.values(),label='b2plyp')
plt.plot(np.array(list(mm.values()))-np.array(list(mmdelta.values())),label='mm after b2plyp')
plt.plot(mm.values(),label='mm final')
plt.ylabel('kJ/mol')
plt.xlabel('conformation')
plt.xticks(range(len(mm)),labels=[ f'{k} : {v}' for k,v in enumerate(mm.keys())],rotation=60)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(rmsd_clqm,label='RMSD between b2plyp input-output')
plt.plot(rmsd_qmmm,label='RMSD between MM relaxation input-output')
plt.ylabel('A')
plt.xlabel('conformation')
plt.xticks(range(len(mm)),labels=[ f'{k} : {v}' for k,v in enumerate(mm.keys())],rotation=60)
plt.legend()
plt.show()